## Objetivos de la clase 📚

El objetivo principal de esta clase es introducirlos a la clasificación de texto en NLP. 
Para esto, implementaremos varios modelos de clasificación destinados a **predecir la categoría de noticias de la radio biobio**.

Los modelos y métodos que usaremos serán los siguientes: 

### 1. **Preprocesamiento**:
El preprocesamiento es el conjunto de técnicas aplicadas a los datos antes de ser utilizados por un modelo de aprendizaje automático. Las técnicas que usaremos son:

- **Tokenización**: Es el proceso de dividir un texto en unidades más pequeñas llamadas *tokens*. Los tokens pueden ser palabras, frases o incluso caracteres individuales, según el nivel de granularidad que se necesite. Por ejemplo, la frase "Hola mundo" se tokeniza como ["Hola", "mundo"].

- **Stemming**: Es una técnica que reduce las palabras a su raíz o forma base. El objetivo es agrupar diferentes formas de una palabra, de modo que puedan tratarse como la misma entidad. Por ejemplo, "corriendo", "corre" y "correr" se reducirían a la raíz "corr".

- **Lematización**: Similar al stemming, pero en lugar de reducir las palabras a su raíz, la lematización convierte las palabras a su forma base o lema según el contexto. Por ejemplo, "correrá" se convierte en "correr". La lematización tiene en cuenta el significado gramatical, lo que la hace más precisa que el stemming.

- **Eliminación de Stop Words**: Las *stop words* son palabras comunes que no aportan mucho significado a la tarea de análisis, como "el", "la", "en", "de". Eliminar estas palabras puede ayudar a reducir el ruido en los datos y mejorar la eficiencia del modelo.

### 2. **Bag of Words**:
El modelo de **Bag of Words (BoW)** es una técnica simple de representación de texto donde un documento o conjunto de palabras se convierte en un conjunto de palabras únicas y su frecuencia dentro de ese documento. Se ignora la gramática y el orden de las palabras, y solo se cuenta cuántas veces aparece cada palabra en el documento. Es un enfoque común para convertir texto en datos que pueden ser usados por modelos de aprendizaje automático. El resultado suele ser una matriz en la que cada fila representa un documento y cada columna representa una palabra única.

### 3. **Clasificador de Bayes**:
El **Clasificador de Bayes** es un algoritmo de clasificación basado en el teorema de Bayes, que calcula la probabilidad de que una instancia pertenezca a una clase dada, en función de sus características. El modelo de Bayes más conocido es el **Naive Bayes**, que asume que las características son independientes entre sí. A pesar de esta suposición ingenua, Naive Bayes es muy eficaz para tareas de clasificación de texto como el filtrado de spam y la categorización de documentos.

### 4. **Logistic Regression (Regresión Logística)**:
La **Regresión Logística** es un modelo de clasificación supervisado que se utiliza para predecir la probabilidad de que una instancia pertenezca a una de dos clases posibles. Aunque se llama "regresión", se utiliza principalmente para problemas de clasificación binaria. La regresión logística estima la probabilidad de que una instancia pertenezca a una clase basándose en una función sigmoide que produce un valor entre 0 y 1. Si el valor es mayor que un umbral (generalmente 0.5), se clasifica la instancia en una clase; de lo contrario, en la otra.

In [292]:
import pandas as pd    
import spacy
import nltk 

from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

from spacy.lang.es.stop_words import STOP_WORDS
nlp = spacy.load("es_core_news_sm", disable=['ner', 'parser', 'tagger'])

In [293]:
#! python -m spacy download es_core_news_sm


Entonces, 

### ¿Qué es la clasificación de texto?

La clasificación de texto consiste en tomar distintos textos y asignarles alguna clase. Dichas clases varían según la task que queramos resolver. Por ejemplo:

    - Detectar emails SPAM -> SPAM, NO SPAM
    - Reviews de Peliculas -> Buena, Mas o menos, Mala, Malísima, Brutalmente mala.
    - Análisis de sentimientos de tweets: Felicidad, Tristeza, Enojo, Ira,...
    - Detectar Fake News -> Es, No es
    - Lenguaje del texto -> Español, Inglés, Chino,...
    - Categoría de una noticia -> Nacional, Internacional, Economía, Sociedad, Opinión...
    - Autor de un texto -> Cada autor es una clase distinta.
Se define formalmente como:

- Input: 

    - Un documento $d$
    - Un conjunto fijo de clases $c_1, c_2, ..., c_j$

- Output: 
    
    - Una clase $c \in C$ para el documento 
    
 
Hay dos clases de métodos para resolver estos problemas: 

1. **Hand-coded Rules 🤙**: 

    Establecemos a mano las reglas que permiten detectar las clases.
    

2. **Supervised Machine Learning 💻**:
   
    Entrenamos clasificadores a partir de muchos ejemplos de documentos etiquetados a mano. 
    

----------------------------------------


## ¿Qué haremos a continuación?


Clasificaremos las noticias de la radio biobio en 20 categorías o tópicos:

```python
[
    'america-latina', 'eeuu', 'europa', 'chile', 'region-metropolitana',
    'region-del-bio-bio', 'negocios-y-empresas', 'region-de-los-lagos',
    'actualidad-economica', 'region-de-valparaiso', 'region-de-la-araucania',
    'curiosidades', 'asia', 'region-de-los-rios', 'entrevistas', 'debates',
    'mediooriente', 'viral', 'animales', 'tu-bolsillo'
]
```

Los pasos a seguir serán: 

1. Primero que nada, descargaremos los datos con los que trabajaremos.

2. Luego, crearemos el sistema mas básico. Este consiste en transformar nuestro texto a `Bag of Words (BoW)` y luego, usar esos vectores para entrenar un clasificador. Este sistema nos puede entregar un muy buen baseline para comenzar a mejorar.

3. Evaluaremos nuestro clasificador según las métricas.

4. A continuación, veremos como mejorar aun mas nuestros resultados. Para esto agregaremos muchas mas técnicas vistas en cátedra, tales como el preprocesamiento de texto y probar con clasificadores aún mas sofisticados.



### Cargar los datasets 


Los datos que usaremos son 5000 documentos con noticias dividas en 20 categorías. Las noticias fueron obtenidas desde la página de la radio biobio.
Cada categoría contiene 250 documentos (noticias). 

Los cargaremos directamente desde un archivo utilizando la librería `pandas` 🐼: 


In [294]:
dataset = pd.read_json('datos/biobio_clean.json')
dataset_r = dataset.copy(deep=True) # respaldo

In [295]:
dataset

author               author_link  \
0               Yerko Roa       /lista/autores/yroa   
1      Valentina González  /lista/autores/vgonzalez   
2          Felipe Delgado   /lista/autores/fdelgado   
3             Matías Vega      /lista/autores/mvega   
4      Valentina González  /lista/autores/vgonzalez   
...                   ...                       ...   
26408      Manuel Stuardo   /lista/autores/mstuardo   
26409      Felipe Delgado   /lista/autores/fdelgado   
26410      Felipe Delgado   /lista/autores/fdelgado   
26411       Nicolás Parra     /lista/autores/nparra   
26412         Emilio Lara      /lista/autores/elara   

                                                   title  \
0      Colapsa otro segmento de casa que se derrumbó ...   
1      Policía busca a mujer acusada de matar a su pa...   
2      Dos detenidos en Liceo de Aplicación: protagon...   
3      Apoyo transversal: Senado aprueba en general p...   
4      Evacuación espontánea en Instituto Nacional po...   
...                                                  ...   
26408  Naciones Unidas abre proceso de postulaciones ...   
26409  Fernando Astengo chocó en estado de ebriedad e...   
26410  Detuvieron a hombre que arrojó combustible a u...   
26411  Revelan identidad de 2 de 6 víctimas fatales e...   
26412  Senado cerrará cuenta paralela al presupuesto ...   

                                                    link  category  \
0      https://www.biobiochile.cl/noticias/nacional/r...  nacional   
1      https://www.biobiochile.cl/noticias/nacional/r...  nacional   
2      https://www.biobiochile.cl/noticias/nacional/r...  nacional   
3      https://www.biobiochile.cl/noticias/nacional/c...  nacional   
4      https://www.biobiochile.cl/noticias/nacional/r...  nacional   
...                                                  ...       ...   
26408  https://www.biobiochile.cl/noticias/nacional/c...  nacional   
26409  https://www.biobiochile.cl/noticias/nacional/r...  nacional   
26410  https://www.biobiochile.cl/noticias/nacional/r...  nacional   
26411  https://www.biobiochile.cl/noticias/nacional/r...  nacional   
26412  https://www.biobiochile.cl/noticias/nacional/c...  nacional   

                subcategory  \
0      region-de-valparaiso   
1      region-metropolitana   
2      region-metropolitana   
3                     chile   
4      region-metropolitana   
...                     ...   
26408                 chile   
26409  region-metropolitana   
26410  region-metropolitana   
26411  region-de-valparaiso   
26412                 chile   

                                                 content  \
0        Noticia en Desarrollo  Estamos recopilando m...   
1        Detectives de la Policía de Investigaciones ...   
2        Dos detenidos fue el saldo de una serie de i...   
3        La sala del Senado aprobó en general el proy...   
4        La mañana de este miércoles se produjo una e...   
...                                                  ...   
26408    Las Naciones Unidas abrió un proceso de post...   
26409    El exfutbolista Fernando Astengo protagonizó...   
26410    Personal de Carabineros detuvo a un hombre q...   
26411    El intendente de Valparaíso, Jorge Martínez,...   
26412    El presidente del Senado, Jaime Quintana (PP...   

                                                    tags  \
0                                                     []   
1      [#parricidio, #PDI, #Pudahuel, #Región Metropo...   
2      [#Incendio, #Liceo de Aplicación, #Región Metr...   
3      [#Inmigración, #Inmigrantes, #Ley, #Migración,...   
4      [#Carabineros, #FFEE, #Gases Lacrimógenos, #In...   
...                                                  ...   
26408  [#cambio climático, #COP25, #Naciones Unidas, ...   
26409  [#Accidente, #Fernando Astengo, #Peñalolén, #R...   
26410  [#Indigente, #Parque Forestal, #Región Metropo...   
26411  [#derrumbe en valparaíso, #Región de Valparaís...   
26412           [#Dipres, #Fiscalía, #Haci

In [296]:
# El número de noticias por clase lo pueden cambiar despues modificando la constante NUM_SAMPLES.
# noten que el número de noticias en el dataset original por categoría está desbalanceada.
# sample intentará sacar la mayor cantidad de ejemplos y retornará siempre, incluso si devuelve 
# menos de los que le pidieron.

NUM_SAMPLES = 250

categorias = dataset['subcategory'].unique()
categorias

array(['region-de-valparaiso', 'region-metropolitana', 'chile',
       'region-de-los-lagos', 'region-del-maule', 'entrevistas', 'tu-voz',
       'america-latina', 'negocios-y-empresas', 'europa',
       'actualidad-economica', 'eeuu', 'africa', 'curiosidades', 'mundo',
       'asia', 'tu-bolsillo', 'mediooriente', 'debate', 'animales',
       'misterios', 'viral', 'oceania', 'consejos', 'salud',
       'region-del-bio-bio', 'region-de-la-araucania',
       'region-de-los-rios', 'region-de-magallanes',
       'region-de-antofagasta', 'region-de-ohiggins', 'region-de-atacama',
       'region-de-tarapaca', 'region-de-aysen', 'region-de-coquimbo',
       'region-de-arica-y-parinacota', 'region-de-nuble', 'videos',
       'educacion-group-nacional', 'viajes', 'fotos-ciudadanas',
       'cronicas', 'test'], dtype=object)

In [297]:
categorias_seleccionadas = [
    'america-latina', 'eeuu', 'europa', 'chile', 'region-metropolitana',
    'region-del-bio-bio', 'negocios-y-empresas', 'region-de-los-lagos',
    'actualidad-economica', 'region-de-valparaiso', 'region-de-la-araucania',
    'curiosidades', 'asia', 'region-de-los-rios', 'entrevistas', 'debates',
    'mediooriente', 'viral', 'animales', 'tu-bolsillo'
]

In [298]:
# Filtrar solo las categorías seleccionadas
dataset_filtrado = dataset[dataset['subcategory'].isin(categorias_seleccionadas)]
dataset_filtrado


author               author_link  \
0               Yerko Roa       /lista/autores/yroa   
1      Valentina González  /lista/autores/vgonzalez   
2          Felipe Delgado   /lista/autores/fdelgado   
3             Matías Vega      /lista/autores/mvega   
4      Valentina González  /lista/autores/vgonzalez   
...                   ...                       ...   
26408      Manuel Stuardo   /lista/autores/mstuardo   
26409      Felipe Delgado   /lista/autores/fdelgado   
26410      Felipe Delgado   /lista/autores/fdelgado   
26411       Nicolás Parra     /lista/autores/nparra   
26412         Emilio Lara      /lista/autores/elara   

                                                   title  \
0      Colapsa otro segmento de casa que se derrumbó ...   
1      Policía busca a mujer acusada de matar a su pa...   
2      Dos detenidos en Liceo de Aplicación: protagon...   
3      Apoyo transversal: Senado aprueba en general p...   
4      Evacuación espontánea en Instituto Nacional po...   
...                                                  ...   
26408  Naciones Unidas abre proceso de postulaciones ...   
26409  Fernando Astengo chocó en estado de ebriedad e...   
26410  Detuvieron a hombre que arrojó combustible a u...   
26411  Revelan identidad de 2 de 6 víctimas fatales e...   
26412  Senado cerrará cuenta paralela al presupuesto ...   

                                                    link  category  \
0      https://www.biobiochile.cl/noticias/nacional/r...  nacional   
1      https://www.biobiochile.cl/noticias/nacional/r...  nacional   
2      https://www.biobiochile.cl/noticias/nacional/r...  nacional   
3      https://www.biobiochile.cl/noticias/nacional/c...  nacional   
4      https://www.biobiochile.cl/noticias/nacional/r...  nacional   
...                                                  ...       ...   
26408  https://www.biobiochile.cl/noticias/nacional/c...  nacional   
26409  https://www.biobiochile.cl/noticias/nacional/r...  nacional   
26410  https://www.biobiochile.cl/noticias/nacional/r...  nacional   
26411  https://www.biobiochile.cl/noticias/nacional/r...  nacional   
26412  https://www.biobiochile.cl/noticias/nacional/c...  nacional   

                subcategory  \
0      region-de-valparaiso   
1      region-metropolitana   
2      region-metropolitana   
3                     chile   
4      region-metropolitana   
...                     ...   
26408                 chile   
26409  region-metropolitana   
26410  region-metropolitana   
26411  region-de-valparaiso   
26412                 chile   

                                                 content  \
0        Noticia en Desarrollo  Estamos recopilando m...   
1        Detectives de la Policía de Investigaciones ...   
2        Dos detenidos fue el saldo de una serie de i...   
3        La sala del Senado aprobó en general el proy...   
4        La mañana de este miércoles se produjo una e...   
...                                                  ...   
26408    Las Naciones Unidas abrió un proceso de post...   
26409    El exfutbolista Fernando Astengo protagonizó...   
26410    Personal de Carabineros detuvo a un hombre q...   
26411    El intendente de Valparaíso, Jorge Martínez,...   
26412    El presidente del Senado, Jaime Quintana (PP...   

                                                    tags  \
0                                                     []   
1      [#parricidio, #PDI, #Pudahuel, #Región Metropo...   
2      [#Incendio, #Liceo de Aplicación, #Región Metr...   
3      [#Inmigración, #Inmigrantes, #Ley, #Migración,...   
4      [#Carabineros, #FFEE, #Gases Lacrimógenos, #In...   
...                                                  ...   
26408  [#cambio climático, #COP25, #Naciones Unidas, ...   
26409  [#Accidente, #Fernando Astengo, #Peñalolén, #R...   
26410  [#Indigente, #Parque Forestal, #Región Metropo...   
26411  [#derrumbe en valparaíso, #Región de Valparaís...   
26412           [#Dipres, #Fiscalía, #Haci

In [299]:
# Obtener los valores únicos y el tamaño de cada subcategoría
valores_unicos = dataset_filtrado['subcategory'].unique()
tamanos_subcategorias = dataset_filtrado.groupby('subcategory').size()

# Crear un DataFrame con las subcategorías y sus tamaños
df_subcategorias = pd.DataFrame({
    'Subcategoría': tamanos_subcategorias.index,
    'Tamaño': tamanos_subcategorias.values
})

# Mostrar el DataFrame
df_subcategorias

Subcategoría  Tamaño
0     actualidad-economica    1040
1           america-latina    4149
2                 animales     425
3                     asia     757
4                    chile    1926
5             curiosidades     796
6                     eeuu    2021
7              entrevistas     663
8                   europa    1976
9             mediooriente     527
10     negocios-y-empresas    1520
11  region-de-la-araucania     875
12     region-de-los-lagos    1345
13      region-de-los-rios     694
14    region-de-valparaiso     901
15      region-del-bio-bio    1631
16    region-metropolitana    1699
17             tu-bolsillo     418
18                   viral     502

In [300]:
# Determinar el número mínimo de ejemplos entre las subcategorías
min_samples = df_subcategorias['Tamaño'].min()

print(f"El número mínimo de ejemplos entre las subcategorías es: {min_samples}")

# Balancear el dataset seleccionando el número mínimo de ejemplos por subcategoría
g = dataset_filtrado.groupby('subcategory')

# Aplicamos el balanceo y nos aseguramos de que el índice se restablezca correctamente
dataset_balanceado = g.apply(lambda x: x.sample(min_samples)).reset_index(drop=True)

# Verificar el tamaño del dataset balanceado y por subcategoría
print(f"Tamaño del dataset balanceado: {len(dataset_balanceado)}")
print("Tamaño por subcategoría después del balanceo:")
print(dataset_balanceado.groupby('subcategory').size())


El número mínimo de ejemplos entre las subcategorías es: 418
Tamaño del dataset balanceado: 7942
Tamaño por subcategoría después del balanceo:
subcategory
actualidad-economica      418
america-latina            418
animales                  418
asia                      418
chile                     418
curiosidades              418
eeuu                      418
entrevistas               418
europa                    418
mediooriente              418
negocios-y-empresas       418
region-de-la-araucania    418
region-de-los-lagos       418
region-de-los-rios        418
region-de-valparaiso      418
region-del-bio-bio        418
region-metropolitana      418
tu-bolsillo               418
viral                     418
dtype: int64


/var/folders/n4/j3zj6_r13rv55gc2c5btztph0000gn/T/ipykernel_42780/3084681870.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dataset_balanceado = g.apply(lambda x: x.sample(min_samples)).reset_index(drop=True)


In [301]:
dataset.sample(10)

author                  author_link  \
6747         Catalina Díaz  /lista/autores/catalinadiaz   
11617          Matías Vega         /lista/autores/mvega   
1160            Diego Vera         /lista/autores/dvera   
12107  César Vega Martínez        /lista/autores/cevega   
963         Manuel Stuardo      /lista/autores/mstuardo   
19972  César Vega Martínez        /lista/autores/cevega   
12584        Gerson Guzmán       /lista/autores/gguzman   
8904        Verónica Reyes     /lista/autores/veroreyes   
17877        Guido Focacci      /lista/autores/gfocacci   
5683           Felipe Díaz         /lista/autores/fdiaz   

                                                   title  \
6747   10 años de cárcel arriesgan integrantes de Los...   
11617  Se judicializa la crisis del PS: denuncias por...   
1160   Fiscal nacional de Perú deja sin efecto destit...   
12107  Berlín pagará indemnización de 2.500 euros a j...   
963    Hundimiento de embarcación en Bahamas deja al ...   
19972  La Fragata Lautaro: detalles del hundimiento e...   
12584  Rescatan a un bebé ruso de edificio destruido ...   
8904   Muertos, evacuados e inundados: huracán de int...   
17877  El peso argentino sigue en caída libre y los e...   
5683   Capturan al presunto autor del homicidio de mu...   

                                                    link       category  \
6747   https://www.biobiochile.cl/noticias/nacional/r...       nacional   
11617  https://www.biobiochile.cl/noticias/nacional/r...       nacional   
1160   https://www.biobiochile.cl/noticias/internacio...  internacional   
12107  https://www.biobiochile.cl/noticias/internacio...  internacional   
963    https://www.biobiochile.cl/noticias/internacio...  internacional   
19972  https://www.biobiochile.cl/noticias/sociedad/m...       sociedad   
12584  https://www.biobiochile.cl/noticias/internacio...  internacional   
8904   https://www.biobiochile.cl/noticias/internacio...  internacional   
17877  https://www.biobiochile.cl/noticias/economia/a...       economia   
5683   https://www.biobiochile.cl/noticias/nacional/r...       nacional   

                subcategory  \
6747    region-de-los-lagos   
11617  region-metropolitana   
1160         america-latina   
12107                europa   
963                   mundo   
19972             misterios   
12584                  asia   
8904                   eeuu   
17877  actualidad-economica   
5683   region-metropolitana   

                                                 content  \
6747     10 Años de cárcel arriesgan dos hombres, mie...   
11617    El diputado socialista Juan Luis Castro anun...   
1160     El fiscal nacional de Perú, Pedro Chávarry, ...   
12107    Alemania pagará una indemnización excepciona...   
963      Autoridades de Bahamas han recuperado hasta ...   
19972    Para muchos marinos de mediados del siglo XX...   
12584    Los socorristas rescataron el martes a un be...   
8904     La tormenta Florence desplegaba su furia en ...   
17877    El peso argentino continuó este jueves su ca...   
5683     La Policía de Investigaciones informó que la...   

                                                    tags  \
6747   [#asaltos, #Banda, #condenas, #detenidos, #Los...   
11617  [#Elecciones, #Juicio, #Partido Socialista, #R...   
1160   [#caso Odebrecht, #José Domingo Pérez, #Perú, ...   
12107    [#Alemania Nazi, #Berlín, #Inglaterra, #judíos]   
963    [#Bahamas, #embarcación, #haitianos, #Hundimie...   
19972  [#Argentina, #Chile, #fragata lautaro, #Incend...   
12584            [#emergencia, #Rescate, #Rusia, #rusos]   
8904   [#Estados Unidos, #FLORENCE, #Huracán, #huracá...   
17877  [#Actualidad Económica, #América Latina, #Arge...   
5683   [#capturan, #mujer quemada en maleta, #Provide...   

                                          embedded_links  publication_datetime  
6747   [https://media.biobiochile.cl/wp-content/uploa...         1557998040000  
11617                                                 [] 

In [302]:
X_train, X_test, y_train, y_test = train_test_split(dataset_balanceado.content,
                                                    dataset_balanceado.subcategory,
                                                    test_size=0.33,
                                                    random_state=42)

In [303]:
X_train

724       El ex subsecretario de Obras Públicas de Arg...
765       El papa Francisco llamó el domingo al diálog...
6900      Por casi una semana los vecinos de Melipilla...
4317      En mayo del presente año, el Banco de Chile ...
4463      El paro de los funcionarios de la ex Superin...
                              ...                        
5226      El intendente de la región de Los Lagos, Har...
5390      Con arresto domiciliario total quedó el adol...
860       Cuando David Fernández (25) ve a un perro mo...
7603      Diversos comentarios generó la decisión toma...
7270      La Empresa Nacional del Petróleo (ENAP) info...
Name: content, Length: 5321, dtype: object

In [304]:
X_test

3286      La académica de la Facultad de Educación de ...
3322      La directora del Sename, Susana Tonda, se re...
5004      El Juzgado de Garantía de Temuco modificó la...
1420      Corea del Norte adoptará a partir de este sá...
3785      Turquía dispone de pruebas, en particular un...
                              ...                        
7552      La tarde de este martes WhatsApp experimentó...
3570      El ministro de Exteriores turco, Mevlüt Çavu...
7395      El Instituto de Previsión Social (IPS) infor...
6174      Funcionarios del Hospital de Quilpué insiste...
3254      Este martes inició el viaje de Michelle Bach...
Name: content, Length: 2621, dtype: object

### Nuestro primer sistema de clasificación


Ahora que tenemos cargado el dataset, podemos implementar nuestro clasificador!

Para esto, usaremos 3 herramientas fundamentales de scikit-learn: un `pipeline`, `CountVectorizer` y `MultinomialNB`.

#### Pipeline 


Un [`pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) es la definición de los procesos que llevará a cabo el sistema que creemos. Nos permite tener unificados todos los procesos a la vez que simplifica el código de nuestro sistema.


En nuestro caso, el pipeline será:

    Dataset -> Bag of Words -> NaiveBayes Clf


#### Bag of Words y CountVectorizer 🎒 


¿Qué era Bag of Words?

Es un modelo en donde transformamos cada una de las oraciones de nuestro dataset en vectores. Cada vector contiene una columna por cada palabra / **token** del vocabulario. Al procesar el dataset, cada oración es mapeada a un vector que cuenta las apariciones de cada una de sus tokens. 

Referencia: [BoW en wikipedia](https://es.wikipedia.org/wiki/Modelo_bolsa_de_palabras)

**Un pequeño ejemplo**

Supongamos que nuestro tokenizador solo separa por espacios.

    - Doc1 : 'I love dogs'
    - Doc2: 'I hate dogs and knitting.
    - Doc3: 'Knitting is my hobby and my passion.

El bag of words quedaría:

`CountVectorizer` es la clase de `scikit` que transformará nuestro texto a Bag of Words. Fijense que es tremendamente útil tenerla dentro de un pipeline ya que fija en un comienzo el vocabulario que tendrá el Bag of Words, evitando discordancias entre los vectores del conjunto de entrenamiento y el de prueba.

#### MultinomialNB

El clasificador

------------------------
#### Creemos el clasificador 🧪

**Primero, definimos el pipeline**

In [305]:
# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer()  

# Definimos el clasificador que usaremos.
clf = MultinomialNB()   

# Creamos el pipeline
text_clf = Pipeline([('vect', vectorizer), ('clf', clf)])

**Luego, lo entrenamos**

In [306]:
# Entrenamos nuestro pipeline
text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('clf', MultinomialNB())])

**Y predecimos**

In [307]:
y_pred = text_clf.predict(X_test)
y_pred

array(['entrevistas', 'entrevistas', 'region-de-la-araucania', ...,
       'tu-bolsillo', 'region-de-valparaiso', 'entrevistas'], dtype='<U22')

**Veamos como nos fue:**

In [308]:
# algunos ejemplos:
pd.DataFrame({'content': X_test, 'category':y_test, 'predicted category': y_pred}) 

content  \
3286    La académica de la Facultad de Educación de ...   
3322    La directora del Sename, Susana Tonda, se re...   
5004    El Juzgado de Garantía de Temuco modificó la...   
1420    Corea del Norte adoptará a partir de este sá...   
3785    Turquía dispone de pruebas, en particular un...   
...                                                 ...   
7552    La tarde de este martes WhatsApp experimentó...   
3570    El ministro de Exteriores turco, Mevlüt Çavu...   
7395    El Instituto de Previsión Social (IPS) infor...   
6174    Funcionarios del Hospital de Quilpué insiste...   
3254    Este martes inició el viaje de Michelle Bach...   

                    category      predicted category  
3286             entrevistas             entrevistas  
3322             entrevistas             entrevistas  
5004  region-de-la-araucania  region-de-la-araucania  
1420                    asia                    asia  
3785            mediooriente            mediooriente  
...                      ...                     ...  
7552                   viral                   viral  
3570                  europa          america-latina  
7395             tu-bolsillo             tu-bolsillo  
6174    region-de-valparaiso    region-de-valparaiso  
3254             entrevistas             entrevistas  

[2621 rows x 3 columns]

In [309]:
# usando la matriz de confusión:

# eje x -> predichos
# eje y -> clase real

print(confusion_matrix(y_test, y_pred))

[[ 97   2   0   0  20   3   4   1   0   0   7   0   0   0   0   0   0  14
    0]
 [  2 117   0   3   1   8  11   0   4   1   0   0   0   0   1   0   0   0
    0]
 [  0   0 109   3   0  13   1   0   6   0   0   0   0   0   0   0   0   0
    8]
 [  1   1   1 112   0  10  13   0   8   6   0   0   0   0   0   0   0   0
    1]
 [  5   2   1   0 100   2   0   3   0   0   2   0   1   0   2   2   5   4
    0]
 [  0   3   6   3   0  99   3   0   8   0   0   0   0   0   0   0   0   4
   10]
 [  1   3   1   5   0  11 102   0   5   8   1   0   0   0   0   0   0   0
    1]
 [  2   1   0   0   5   0   0 123   0   2   0   0   0   0   1   0   2   2
    1]
 [  0   5   1   2   0   3   2   0 100   4   0   0   0   0   0   0   0   0
    1]
 [  0   1   0   3   0   5   2   0   3 128   0   0   0   0   0   0   0   0
    0]
 [ 38   0   0   0   9   6   1   0   0   0  70   0   0   0   1   0   0  29
    0]
 [  3   1   0   0  31   0   0   1   0   0   1  69  11   7   6   1  13   2
    1]
 [  3   0   0   0  16   2   

#### Métricas de Evaluación

Las métricas definen un puntaje de evaluación que indica que tal le fue al sistema. Hay muchas formas distintas de medir su rendimiento. Entre estas, tenemos:

- `precision`: El número de documentos de una clase clasificados correctamente dividido por el número de documentos totales clasificados como esa clase.

- `recall`: El número de documentos de una clase clasificados correctamente dividido por el número de los documentos que se deberían haber clasificado como esa clase.(número de documentos reales de esa clase).

- `f1-score` : Es la media armónica entre los anteriores.

- `accuracy` : La cantidad de documentos clasificados correctamente versus todos los documentos

Por otra parte, tenemos dos formas de ver dichas métricas agrupadas:

- `Macroaveraging`:    Se computan las métricas por cada clase y luego de promedia.

- `Microaveraging`:    Se recolectan las clasificaciones por cada clase, se computa la tabla de contingencia (todos los elementos clasificados) y se evalua. Representa un Macroaveraging ponderado por el número de miembros de una clase.

In [310]:
# usando el classification report:
print(classification_report(y_test, y_pred))

                        precision    recall  f1-score   support

  actualidad-economica       0.53      0.66      0.59       148
        america-latina       0.85      0.79      0.82       148
              animales       0.83      0.78      0.80       140
                  asia       0.82      0.73      0.77       153
                 chile       0.39      0.78      0.52       129
          curiosidades       0.42      0.73      0.53       136
                  eeuu       0.73      0.74      0.74       138
           entrevistas       0.88      0.88      0.88       139
                europa       0.73      0.85      0.78       118
          mediooriente       0.86      0.90      0.88       142
   negocios-y-empresas       0.76      0.45      0.57       154
region-de-la-araucania       0.97      0.47      0.63       147
   region-de-los-lagos       0.81      0.62      0.70       138
    region-de-los-rios       0.90      0.66      0.76       141
  region-de-valparaiso       0.67      

#### Ejecutemos algunas consultas!

In [311]:
text_clf.predict([
    ("En puerto montt se encontró un perrito, que aparentemente,"
    "habría consumido drogas de alto calibre. Producto de esto,"
    "se ponostíca que padecerá severa caña durante varios dias.")
])

array(['region-de-los-lagos'], dtype='<U22')

In [312]:
text_clf.predict(["kim jong un será el próximo candidato a ministro de educación."])

array(['asia'], dtype='<U22')

In [313]:
text_clf.predict([("El banco mundial presentó para chile un decrecimiento"
                   "económico de 92% y una inflación de 8239832983289%.")])

array(['actualidad-economica'], dtype='<U22')

-------------------------

Se ven bastante buenos los resultados. ¿Pero, podremos mejorarlos?

### Preprocesamiento del texto

Podemos preprocesar los textos?, para intentar mejorar.
Es decir, cómo hacemos el proceso de tokenización (separación de las palabras).



Alguna de las técnicas son:


- Eliminación de Stopwords
- Stemming
- Lematización

Existen otros preprocesadores que agregan información a las oraciones, tales como aquellos que indican negaciones.

A continuación, describiremos con mas detalle cada uno de estas técnicas.

#### Tokenizar ➗

¿Qué era tokenizar?


    Es el proceso de convertir una secuencia de carácteres (por ejemplo, una oración) en una secuencia de valores distintos entre si llamados tokens.
    
Referencia: [Tokenización en wikipedia](https://en.wikipedia.org/wiki/Lexical_analysis#Tokenization)



**spaCy y el objeto nlp**

`nlp` es el objeto que nos permite usar e interactuar con la librería [`spacy`](https://spacy.io/).
Esta librería incluye variadas herramientras, tales como tokenizar, lematizar, descartar stopwords, entre otras (para este auxiliar, solo utilizaremos las mencionadas). El objeto nlp lo instanciamos en la sección de imports.

Para usarla, simplemente se le pasa el texto como parámetro, como veremos en el siguiente ejemplo: 

In [314]:
DOC = "hermanito mio te estas pegando el show"

tokens = []
for word in nlp(DOC):
    tokens.append(word)

tokens

[hermanito, mio, te, estas, pegando, el, show]

**Observación**: Para este auxiliar usaremos `List Comprehension`, otra forma de hacer un for un poco mas reducida.
Una muy buena referencia de esto [aquí](https://www.programiz.com/python-programming/list-comprehension).

La operación anterior usando esta sintáxis quedaría como:

In [315]:
tokens = [word for word in nlp(DOC)]
tokens

[hermanito, mio, te, estas, pegando, el, show]

#### Stopwords 🛑

¿Qué eran las stopwords?

    Las Stopwords son palabras muy comunes en nuestro lenguaje y que por lo tanto, no aportan mucha información. Existen múltiples listas de stopwords para muchos idiomas y la aplicación de estas variará caso a caso.

    
Referencias: [Stopwords en Wikipedia](https://en.wikipedia.org/wiki/Stop_words)

En este caso, utilizaremos las stopwords inlcuidas en la librería spaCy en español

In [316]:
print(len(STOP_WORDS))
print(list(STOP_WORDS)[0:20])

521
['seis', 'vuestra', 'algún', 'sus', 'debajo', 'saber', 'mías', 'lado', 'o', 'tenido', 'antes', 'da', 'temprano', 'podría', 'podriais', 'suyo', 'través', 'ti', 'acuerdo', 'cuál']


#### Stemming 🔪

¿Qué era el stemming? 

    Son un conjunto de métodos enfocados en reducir cada palabra a su raiz.

Referencia: [Stemming en Wikipedia](https://en.wikipedia.org/wiki/Stemming)
  
**Ejemplos: **


| word | stem of the word  |
|---|---|
working | work
worked | work
works | work

**nltk**

En este caso, utilizaremos la segunda librería de herramientas de nlp: [`nltk`](https://www.nltk.org/). Esta provee una buena herramienta para hacer stemming en español : `SnowballStemmer`

In [317]:
stemmer = SnowballStemmer('spanish')
stemmed_doc = [stemmer.stem(str(token)) for token in tokens]
print(stemmed_doc)

['hermanit', 'mio', 'te', 'estas', 'peg', 'el', 'show']


#### Lematización 🙀

¿Qué era lematización? 

    Es el proceso de transformar cada token a su lema, el cual es la palabra base sin ningún tipo de flexión o alteración como las conjugaciones, por ejemplo.
    
    

  
    
    
Referencia: [Lematización en wikipedia](https://en.wikipedia.org/wiki/Lemmatisation)

Refernecia: [Flexión en las palabras](https://es.wikipedia.org/wiki/Flexi%C3%B3n_(ling%C3%BC%C3%ADstica))

**Ejemplos**

| word | lemma  |
|---|---|
dije| decir 
guapas | guapo
mesa | mesas


 <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/29/Flexi%C3%B3nGato-svg.svg/300px-Flexi%C3%B3nGato-svg.svg.png" alt="Flexión de gato" style="width: 200px;"/>
 

**Lematizar el texto**

Al igual que la tokenización, utilizaremos `scpaCy` (a través del objeto `nlp`) para lematizar el contenido.

In [318]:
lemmatized_content = [word.lemma_ for word in nlp(DOC)]
print(lemmatized_content)

['hermanito', 'mio', 'tú', 'este', 'pegar', 'el', 'show']


**Discusión:**

    ¿Cuál es mejor?

### Sistema de clasificación con preprocesamiento

Para agregar los tokenizadores en el sistema, creamos funciones que que cada documento de forma individual usando nuestro preprocesador favorito. Luego, `CountVectorizer` se encargará de usar estas funciones sobre todo el dataset.

In [319]:
# Tokenizers para CountVectorizer

# Solo tokenizar el doc usando spacy.
def tokenizer(doc):
    return [x.orth_ for x in nlp(doc)]


# Tokenizar y remover las stopwords del doc
def tokenizer_with_stopwords(doc):
    return [x.orth_ for x in nlp(doc) if x.orth_ not in STOP_WORDS]


# Tokenizar y lematizar.
def tokenizer_with_lemmatization(doc):
    return [x.lemma_ for x in nlp(doc)]

# Tokenizar y hacer stemming.
def tokenizer_with_stemming(doc):
    stemmer = SnowballStemmer('spanish')
    return [stemmer.stem(word) for word in [x.orth_ for x in nlp(doc)]]

#### Creamos nuestro clasificador


**Definimos el pipeline**

In [320]:
# seleccionamos el tokenizador a usar:
TOKENIZER = tokenizer_with_stemming

# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word',
                             tokenizer=TOKENIZER,
                             ngram_range=(1, 1))

# Definimos el clasificador que usaremos.
clf = MultinomialNB()   

# Creamos el pipeline
text_clf_2 = Pipeline([('vect', vectorizer), ('clf', clf)])

**Entrenamos nuestro pipeline y predecimos**


In [321]:
text_clf_2.fit(X_train, y_train)
y_pred = text_clf_2.predict(X_test)

/Users/pavt/Documents/personal/ufro/2024-2/Ing-datos-civil/id-civil/.id_civil/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [322]:
# usando la matriz de confusión:
print(confusion_matrix(y_test, y_pred),
      '\n\n-------------------------------------------------------\n')
# usando el classification report:
print(classification_report(y_test, y_pred))

[[ 98   2   0   1  19   3   1   2   0   0   8   0   0   0   0   0   0  14
    0]
 [  2 117   0   4   1   7  11   0   4   1   0   0   0   0   0   0   0   0
    1]
 [  0   0 110   3   0  12   1   0   5   0   0   0   0   0   0   0   0   0
    9]
 [  1   0   1 116   0  10  10   0   7   7   0   0   0   0   0   0   0   0
    1]
 [  6   2   0   0  96   2   0   3   0   0   3   1   1   0   1   3   6   4
    1]
 [  0   4   7   3   0 102   4   0   6   0   0   0   0   0   0   0   0   3
    7]
 [  1   5   1   5   1   8  99   0   5  10   1   0   0   0   0   0   0   0
    2]
 [  1   1   0   0   6   1   0 124   0   2   0   0   1   0   0   0   2   1
    0]
 [  0   5   1   2   0   4   2   0  97   5   1   0   0   0   0   0   0   0
    1]
 [  0   1   0   3   0   4   1   0   3 130   0   0   0   0   0   0   0   0
    0]
 [ 34   0   0   0  10   5   1   2   0   0  74   0   0   0   1   0   1  26
    0]
 [  2   0   0   0  31   1   0   0   0   0   3  69  12   7   6   1  13   2
    0]
 [  4   0   0   0  14   2   

#### Pregunta abierta: ¿Por qué no mejoran los resultados?

[Aquí]("https://www.quora.com/Is-it-normal-to-get-better-accuracy-without-stemming-and-lemmatization-than-using-them-in-NLP-text-classification") hay una muy buena discusión al respecto.

### Clasificación usando Regresión Logísitica

No profundizaremos en este clasificador, mas del hecho de que se "supone" que debería tener mejor rendimiento que el de bayes.

Referencia: [Regresión Logística](https://en.wikipedia.org/wiki/Logistic_regression)

**Definimos nuestro Pipeline**

In [325]:
# Qué tokenizer usaremos?
TOKENIZER = tokenizer_with_lemmatization

# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word',
                             tokenizer=TOKENIZER,
                             ngram_range=(1, 1))

# Ahora definimos regresión logística como clasificador.
log_mod = LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000)   
log_pipe = Pipeline([('vect', vectorizer), ('clf', log_mod)])

**Entrenamos y predecimos**

In [326]:
log_pipe.fit(X_train, y_train)
y_pred = log_pipe.predict(X_test)

/Users/pavt/Documents/personal/ufro/2024-2/Ing-datos-civil/id-civil/.id_civil/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/pavt/Documents/personal/ufro/2024-2/Ing-datos-civil/id-civil/.id_civil/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


**Evaluamos**

In [327]:
# usando la matriz de confusión:
print(confusion_matrix(y_test, y_pred),
      '\n\n-------------------------------------------------------\n')
# usando el classification report:
print(classification_report(y_test, y_pred))

[[100   3   0   1   7   1   1   1   0   0  17   0   0   0   1   3   0  13
    0]
 [  1 116   1   3   2   3  11   0   5   0   1   0   0   0   1   0   2   0
    2]
 [  0   0 112   3   0   1   1   0   4   0   0   0   0   0   0   0   1   0
   18]
 [  2   0   2 118   2   7   9   0   2   6   1   0   0   0   0   0   0   0
    4]
 [  8   0   0   0  80   2   0   4   0   0   4   3   1   1   8   5  10   2
    1]
 [  0   3   7   5   0  78   7   0   9   1   1   0   0   0   1   0   1   6
   17]
 [  1   4   1   7   3   8  99   0   5   5   1   0   0   0   1   1   0   1
    1]
 [  0   1   0   0   1   0   0 135   0   0   0   1   0   1   0   0   0   0
    0]
 [  1   4   2   7   0   2   2   0  92   3   3   0   0   0   1   0   0   0
    1]
 [  1   2   1   4   0   6   8   0   3 117   0   0   0   0   0   0   0   0
    0]
 [ 17   0   1   0   3   1   1   1   0   0 102   0   0   1   2   5   2  17
    1]
 [  0   0   0   0   3   0   0   1   0   0   1 126   3   5   0   2   5   0
    1]
 [  0   0   0   0   2   0   

### N-gramas

Los n-gramas son conjuntos de n-tokens seguidos entre si. La idea de usar esto es que además, capturemos conceptos. 

Por ejemplo, si usamos 2-gramas sobre `'Hoy día comí lentejas'`, esta quedaría como:


```python
['hoy dia', 'día comí', 'comí lentejas']
```

`CountVectorizer` tiene la opción para poner n-gramas del tamaño que tu quieras, y además incluir mas pequeños. Todo esto se define en el parámetro `ngram_range`. Este recibe una tupla con los rangos del n-grama mas pequeño y el mas grande. Por ejemplo, para (1,2), la oración anterior quedaría: 


```python
['hoy', 'día', 'comí', 'lentejas', 'hoy dia', 'día comí', 'comí lentejas']
```

Nota que esto incrementa el tamaño de los vectores de Bag of words y por lo tanto, del entrenamiento y de la predicción. 

In [328]:
# Qué tokenizer usaremos?
TOKENIZER = tokenizer_with_lemmatization

# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word',
                     
                             tokenizer=TOKENIZER,
                             ngram_range=(1, 3))

# Ahora definimos regresión logística como clasificador.
log_mod = LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000)   
log_pipe = Pipeline([('vect', vectorizer), ('clf', log_mod)])

**Entrenamos y predecimos**

In [ ]:
# usando la matriz de confusión:
print(confusion_matrix(y_test, y_pred),
      '\n\n-------------------------------------------------------\n')
# usando el classification report:
print(classification_report(y_test, y_pred))

### Bonus: Clasificación de Autoría de documentos

¿Existirá un patrón en como escriben los periodistas que nos permitan identificarlos a partir de sus textos?

In [329]:
dataset_r.author.value_counts()[0:20]

author
Diego Vera               4471
Emilio Lara              1380
Matías Vega              1230
César Vega Martínez      1202
María José Villarroel    1157
Gonzalo Cifuentes        1122
Manuel Stuardo           1022
Manuel Cabrera            999
Valentina González        966
Nicole Briones            850
Felipe Delgado            825
Yessenia Márquez          764
Paola Alemán              760
Verónica Reyes            718
Jonathan Flores           618
Nicolás Díaz              592
Sebastián Asencio         544
Catalina Díaz             523
Ariela Muñoz              515
Nicolás Parra             503
Name: count, dtype: int64

In [330]:
NUM_SAMPLES = 250

def process_datasets_by_author(dataset):
    
    # creamos una nueva columna titulo y contenido.
    content = dataset['title'] + '. ' + dataset['content'] 
    # obtenemos las clases
    subcategory = dataset.author
    # dejamos en el dataset solo contenido de la noticia y categoria
    dataset = pd.DataFrame({'content': content, 'author': subcategory})

    selected_authors = ['Diego Vera', 'Emilio Lara', 'Matías Vega', 'César Vega Martínez',
           'María José Villarroel', 'Gonzalo Cifuentes', 'Manuel Stuardo',
           'Manuel Cabrera', 'Valentina González', 'Nicole Briones',
           'Felipe Delgado', 'Yessenia Márquez', 'Paola Alemán', 'Verónica Reyes',
           'Jonathan Flores', 'Nicolás Díaz', 'Sebastián Asencio', 'Catalina Díaz',
           'Ariela Muñoz', 'Nicolás Parra']

    # filtrar solo categorias seleccionadas
    dataset = dataset[dataset['author'].isin(selected_authors)]

    # balancear clases
    g = dataset.groupby('author')
    dataset = pd.DataFrame(
        g.apply(lambda x: x.sample(NUM_SAMPLES).reset_index(drop=True))).reset_index(
            drop=True)
    
    return dataset

    



In [332]:
author_dataset = process_datasets_by_author(dataset_r.copy(deep=True))

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(
    author_dataset.content,
    author_dataset.author,
    test_size=0.33,
    random_state=42)

/var/folders/n4/j3zj6_r13rv55gc2c5btztph0000gn/T/ipykernel_42780/4182984880.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  g.apply(lambda x: x.sample(NUM_SAMPLES).reset_index(drop=True))).reset_index(


In [333]:
author_dataset

content            author
0     Niña migrante de 7 años muere de deshidratació...      Ariela Muñoz
1     Guaidó participará del Grupo de Lima y conside...      Ariela Muñoz
2     Mallplaza tiene por primera vez en su historia...      Ariela Muñoz
3     A dos meses del incendio: así se viven las cer...      Ariela Muñoz
4     Mujer postrada muere en un incendio al interio...      Ariela Muñoz
...                                                 ...               ...
4995  Dejan a hombre herido de bala en Hospital de V...  Yessenia Márquez
4996  Código Azul en La Araucanía: 249 personas vive...  Yessenia Márquez
4997  Fonasa asegura que perdidas por cambio de usua...  Yessenia Márquez
4998  Madre de joven desaparecido hace 7 meses en Hu...  Yessenia Márquez
4999  Pescadores artesanales realizan corte de ruta ...  Yessenia Márquez

[5000 rows x 2 columns]

#### Definir el Pipeline

In [334]:
# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word',
                             ngram_range=(1, 1))

# Definimos el clasificador. Usaremos bayes, ya que regresión logística se demora 1/4 del tiempo del universo.
clf = MultinomialNB()   


# Creamos el pipeline
log_pipe_by_author = Pipeline([('vect', vectorizer), ('clf', clf)])

#### Entrenar

In [335]:
log_pipe_by_author.fit(X_train_2, y_train_2)
y_pred = log_pipe_by_author.predict(X_test_2)

#### Evaluar

In [336]:
# usando la matriz de confusión:
print(confusion_matrix(y_test_2, y_pred),
      '\n\n-------------------------------------------------------\n')
# usando el classification report:
print(classification_report(y_test_2, y_pred))

[[ 1  2  0  6  0 11  0 27  1  0  9  9  0  4  0  6  1  1 14  2]
 [ 0 35  0  1  0  2  0  3  2  9  9  0  2 16  2  1  0  0  1  2]
 [ 0  0 56  5  0  0  0  2  0  0  0  0  0  0  0 26  0  0  1  0]
 [ 0  0  1 54  0  0  0  5  0  0  0  0  0  0  0  7  3  0  2  0]
 [ 0  3  1 33  4  3  1 19  1  0  5  2  1  7  0 11  1  0  4  0]
 [ 0  0  0 12  0 36  0  3  0  2  3  6  0  2  1  7  0  2  3  0]
 [ 0  2  0 28  0  2  3 18  1  0  2  6  1  7  1  9  3  0  9  0]
 [ 0  0  0 12  0  1  0 45  0  0  2  4  0  1  0  6  0  0  6  0]
 [ 0  2  0  8  0  3  0  8 12  2  4  2  1 21  9  6  1  1  1  1]
 [ 0  7  0 12  0  1  0  7  1 11 15  1  5 15  1  2  1  0  0  6]
 [ 1  2  0  6  0  7  0  4  1  2 38  0  1  5  1  3  1  2  2  1]
 [ 0  1  0 26  1  3  0 23  1  0  4  6  1  5  2  3  1  2  4  0]
 [ 0  7  0  5  0  1  0  5  4  6 14  1 10 19  3  1  0  0  0 11]
 [ 0  2  1  2  0  6  0  8  1  0  3  3  1 41  2  0  1  0  1  1]
 [ 0  2  0 10  0  1  0 12  9  0  6  1  2  9 18  6  0  0  3  1]
 [ 0  0  1 28  0  0  0  2  0  0  0  0  0  0  0 54  0  0

## Referencias
 

Slides:
- https://web.stanford.edu/~jurafsky/slp3/slides/7_NB.pdf


Análisis de sentimientos como clasificación de texto:
- https://affectivetweets.cms.waikato.ac.nz/benchmark/

Algunos Recursos útiles
- [Pandas Cheat Sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)
- [Scikit-learn Cheat Sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Scikit_Learn_Cheat_Sheet_Python.pdf)
- [Spacy Tutorial](https://www.datacamp.com/community/blog/spacy-cheatsheet)
- [NLTK Cheat sheet](http://sapir.psych.wisc.edu/programming_for_psychologists/cheat_sheets/Text-Analysis-with-NLTK-Cheatsheet.pdf)